In [1]:
import torch
import argparse
import time
import math
import os, sys
import itertools
import pickle
from tqdm import tqdm
import numpy as np
import os.path as osp

In [2]:
torch.cuda.device_count()

1

In [3]:
def loadpkl(path):
    with open(path, 'rb') as f:
        rst=pickle.load(f)
    return rst

In [4]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/full_data_no_embed.pkl"
full_data=loadpkl(path)

In [5]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/test_result/eval_key_id2embed.pkl"
key_embedding=loadpkl(path)

In [6]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_dev_pred_1000.pkl"
pred_set=loadpkl(path)

In [7]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_dev_gold.pkl"
gold_set=loadpkl(path)

In [8]:
query_id2embedding=key_embedding
key_id2embedding=key_embedding

In [10]:
candidate_set={}
candidate_ref_set={}
for k,v in pred_set.items():
    cur=set()
    for paper in v[:200]:
        cur.add(paper)
        refset=set()
        for ref in full_data[paper]["references"]:
            cur.add(ref["paperId"])
            refset.add(ref["paperId"])
        candidate_ref_set[paper]=refset
    candidate_set[k]=list(cur)

In [11]:
print(len(candidate_ref_set))

133604


In [23]:
def rerank(candidate_set,candidate_ref_set,full_data,rerank_num=None):
    query=list(candidate_set.keys())
    rerank_candidate={}
    if rerank_num is None:
        for v in candidate_set.values():
            if rerank_num is None or rerank_num>len(v):
                rerank_num=len(v)
    pbar = tqdm(range(0,len(candidate_set),1), postfix=f"calculating f1")
    for i in pbar:
        query_paper=query[i]
        cite_frequency={}
        for candidate in candidate_set[query_paper]:
            if candidate in candidate_ref_set.keys():
                # this is possibly the anchor paper
                for ref in list(candidate_ref_set[candidate]):
                    # deal with the ref of an anchor paper
                    if ref in candidate_set[query_paper]:
                        if ref not in cite_frequency.keys():
                            cite_frequency[ref]=1
                        else:
                            cite_frequency[ref]+=1
                    else:
                        # this is not anchor paper for this query
                        break
            if candidate not in cite_frequency.keys():
                cite_frequency[candidate]=1
            else:
                cite_frequency[candidate]+=1
                
        top_k = sorted(cite_frequency.items(), key=lambda x:x[1], reverse=True)
        rerank = [k for k,v in top_k[:rerank_num]]
        rerank_candidate[query_paper]=rerank
    print("min rerank num->",rerank_num)
    return rerank_candidate
        
        


In [24]:
rerank_candidate=rerank(candidate_set,candidate_ref_set,full_data,rerank_num=None)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 879/879 [03:34<00:00,  4.10it/s, calculating f1]

min rerank num-> 1510


In [25]:
sys.path.append("/share/data/mei-work/kangrui/github/ssref")
from utils.test_utils import *
from utils.utils import *

In [28]:
def show_rerank_rst(rerank_candidate,gold_set):
    pool_length=len(next(iter(rerank_candidate.values())))
    i=2
    while 1:
        all_pred_topN=[]
        all_gold=[]
        for query_paper,paperlist in rerank_candidate.items():
            topN=paperlist[:i]
            all_pred_topN.append(topN)
            all_gold.append(gold_set[query_paper])
        all_pred_topN=np.array(all_pred_topN)
        metric = get_precision_recall_f1(all_pred_topN, all_gold)
        print(f"top{i}:\n{metric}")
        if i>pool_length:
            break
        i*=2

In [29]:
show_rerank_rst(rerank_candidate,gold_set)

top2:
{'precision': 0.2571103526734926, 'recall': 0.012718064153066966, 'f1': 0.024237224516059842, 'true_pos': 452, 'false_pos': 1306, 'false_neg': 35088, 'num_sample': 879}
top4:
{'precision': 0.2966439135381115, 'recall': 0.02934721440630276, 'f1': 0.05341048750512085, 'true_pos': 1043, 'false_pos': 2473, 'false_neg': 34497, 'num_sample': 879}
top8:
{'precision': 0.2858361774744027, 'recall': 0.056555993247045584, 'f1': 0.09442826270788313, 'true_pos': 2010, 'false_pos': 5022, 'false_neg': 33530, 'num_sample': 879}
top16:
{'precision': 0.24360068259385664, 'recall': 0.0963984243106359, 'f1': 0.13813402144988307, 'true_pos': 3426, 'false_pos': 10638, 'false_neg': 32114, 'num_sample': 879}
top32:
{'precision': 0.19094852104664392, 'recall': 0.15112549240292628, 'f1': 0.16871897970723124, 'true_pos': 5371, 'false_pos': 22757, 'false_neg': 30169, 'num_sample': 879}
top64:
{'precision': 0.1394517918088737, 'recall': 0.2207371975239167, 'f1': 0.17092248028236523, 'true_pos': 7845, 'false_